In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import seaborn as sns
from pyDOE import lhs

import os
import time

plt.rcParams.update({'font.size':18})

In [2]:
layers = [3] + [30]*10 + [1]
class MLP(nn.Module):
    def __init__(self, mlp_layers):
        super(MLP, self).__init__()
        
        self.model = nn.Sequential()
        for i in range(len(mlp_layers)-2):
            layer = nn.Sequential()
            layer.add_module(f'fc{i}', nn.Linear(mlp_layers[i], mlp_layers[i+1], bias=True))
            layer.add_module(f'act{i}', nn.Tanh())
            self.model.add_module(f'layer{i}', layer)

        last_layer = nn.Sequential()
        last_layer.add_module(f'fc{len(mlp_layers)-2}', nn.Linear(mlp_layers[-2], mlp_layers[-1], bias=False))
        self.model.add_module(f'layer{len(mlp_layers)-2}', last_layer)
        
#         for param in self.parameters():
#             if len(param.shape) > 1:
#                 nn.init.kaiming_normal_(param)
    
    def forward(self, X):
        return self.model(X)

    
model_psi = MLP(layers)
model_p = MLP(layers)
model_phil = MLP(layers)
# model

In [4]:
# Load Data
data = scipy.io.loadmat('../Data/MultiBubbleData_TDN.mat')
# tdata = scipy.io.loadmat('../Data/t.mat')
# pdata = scipy.io.loadmat('../Data/p.mat')
# udata = scipy.io.loadmat('../Data/u.mat')
# vdata = scipy.io.loadmat('../Data/v.mat')
# phildata = scipy.io.loadmat('../Data/phi.mat')
print(data.keys())
u_star = data['u'] # N x T
v_star = data['v'] # N x T
P_star = data['p'] # N x T
Phil_star = data['phil'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = u_star # U_star[:,0,:] # N x T
VV = v_star # U_star[:,1,:] # N x T
PP = P_star # N x T
Phil = Phil_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1
phil = Phil.flatten()[:,None] # NT x 1

X_psi = np.concatenate([x, y, t], axis=1)
X_p = np.concatenate([x, y, t], axis=1)
X_phil = np.concatenate([x, y, t], axis=1)

# Training Data
# 3/4 of the data is for training 
idx = np.random.choice(N*T, int(N*T*0.75), replace=False)
x_train = x[idx,:] # [idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]
p_train = p[idx,:]
phil_train = phil[idx,:]
X_train_psi = np.concatenate([x_train, y_train, t_train], axis=1)
X_train_p = np.concatenate([x_train, y_train, t_train], axis=1)
X_train_phil = np.concatenate([x_train, y_train, t_train], axis=1)

print(X_train_psi.shape)
print(X_psi.shape)
u.shape[0]-u_train.shape[0]

def get_lb_ub(X_res):
    x = X_res[:,[0]]
    y = X_res[:,[1]]
    t = X_res[:,[2]]
#     lb = np.array([x.min(), y.min(), t.min()])
    lb = torch.tensor([x.min(), y.min(), t.min()])
#     ub = np.array([x.max(), y.max(), t.max()])
    ub = torch.tensor([x.max(), y.max(), t.max()])
    return lb, ub
X_star

t_train = t_train.astype(x_train.dtype)
t_train.dtype

dict_keys(['__header__', '__version__', '__globals__', 'X_star', 'p', 'phil', 't', 'u', 'v'])
(285274, 3)
(380366, 3)


dtype('<f8')

In [5]:
def grad(outputs, inputs):
    return torch.autograd.grad(outputs, inputs,
                               grad_outputs=torch.ones_like(outputs),
                               create_graph=True)

In [6]:
class PINN(nn.Module):
    def __init__(self, model_psi, model_p, model_phil, mu=None, sigma=None):
        super(PINN, self).__init__()
        self.model_psi = model_psi
        self.model_p = model_p
        self.model_phil = model_phil
        
        if mu is not None and sigma is not None:
            self.is_inputs_normalization = True
            self.mu = mu
            self.sigma = sigma
            print(f'forward with normalization, mu={self.mu.tolist()}, sigma={self.sigma.tolist()}')
        else:
            self.is_inputs_normalization = False
    
    def forward(self, X_res_psi, X_res_p, X_res_phil,  u_data, v_data, p_data, phil_data):
        u_pred, v_pred = self.net_psi(X_res_psi)
        loss_u = torch.mean((u_pred-u_data)**2)
        loss_v = torch.mean((v_pred-v_data)**2)
        loss_p = torch.mean((self.net_p(X_res_p)-p_data)**2)
        loss_phil = torch.mean((self.net_phil(X_res_phil)-phil_data)**2)
        loss = loss_u + loss_v + loss_p + loss_phil
        return loss, loss_u, loss_v, loss_p, loss_phil
    
    def net_psi(self, X):
        if self.is_inputs_normalization == True:
            X = (X - self.mu) / self.sigma
        X.requires_grad_(True)
        psi = self.model_psi(X)
        
        grad_psi = grad(psi, X)[0]
        
        u_pred = grad_psi[:,[1]]
        v_pred = -grad_psi[:,[0]]

        return u_pred, v_pred
    
    def net_p(self, X):
        if self.is_inputs_normalization == True:
            X = (X - self.mu) / self.sigma
        return self.model_p(X)
    
    def net_phil(self, X):
        if self.is_inputs_normalization == True:
            X = (X - self.mu) / self.sigma
        return self.model_phil(X)
    

pinn = PINN(model_psi, model_p, model_phil)

In [8]:
model_path = r'./model'
model_state = torch.load(os.path.join(model_path, 'pinn_lbfgs.pth'))
pinn.model_psi.load_state_dict(model_state['model_psi_state'])
pinn.model_p.load_state_dict(model_state['model_p_state'])
pinn.model_phil.load_state_dict(model_state['model_phil_state'])

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.